#### This is implemented basis this [tutorial](https://langchain-opentutorial.gitbook.io/langchain-opentutorial/17-langgraph/01-core-features/12-langgraph-conversation-summaries), with custom reducer and checkpoint saver


In [18]:
from langgraph_dynamodb_checkpoint import DynamoDBSaver
from typing import Literal, Annotated
from langgraph_utils import call_model
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph_reducer import MessagePrunerNode , PrunableStateFactory

In [2]:
saver = DynamoDBSaver(
    table_name="reducer_test2",
    max_read_request_units=50,  # Optional, default is 100
    max_write_request_units=50,  # Optional, default is 100
    ttl_seconds=86400
)

Table 'reducer_test2' already exists.


In [3]:
# We'll store both the conversation messages and the summary in the state
class State(MessagesState):
    messages: Annotated[list, add_messages]
    summary: str

In [4]:
def should_continue(state: State) -> Literal["summarize_conversation", END]:
    """
    Check if the conversation is too long (over 6 messages).
    If it is, move to the 'summarize_conversation' node.
    Otherwise, end the conversation.
    """
    messages = state["messages"]
    if len(messages) > 6:
        return "summarize_conversation"
    return END

In [5]:
def ask_llm(state: State):
    """
    If a summary of the conversation already exists, we include it as a
    system message. Otherwise, we just use the existing messages.
    """
    summary = state.get("summary", "")

    if summary:
        system_message = f"This is the conversation summary so far: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]

    response = call_model("gpt-4o", "openai", messages)
    return {"messages": [response]}

In [6]:
from functools import partial

# Create a partially-applied function
model_func = partial(call_model, "gpt-4o", "openai")

In [8]:
pruner_node = MessagePrunerNode(min_messages=4, max_messages=6, model_func=model_func)

In [9]:
# Build the workflow graph
workflow = StateGraph(State)

# Add nodes
workflow.add_node("conversation", ask_llm)
workflow.add_node("summarize_conversation",pruner_node)

# Define edges
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile with memory checkpoint
app = workflow.compile(checkpointer=saver)

In [10]:
def print_update(update):
    """
    Helper function to print out updates during streaming.
    """
    for k, v in update.items():
        for m in v.get("messages", []):
            m.pretty_print()
        if "summary" in v:
            print(v["summary"])

In [13]:
# Initialize a configuration object with thread ID
config = {"configurable": {"thread_id": "3"}}

# 1) First user message
input_message = HumanMessage(content="Hello! Nice to meet you. My name is Junseong.")
input_message.pretty_print()

# Process the first message in streaming mode and print updates
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

# 2) Second user message
input_message = HumanMessage(content="Do you remember my name?")
input_message.pretty_print()

# Process the second message in streaming mode and print updates
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

# 3) Third user message
input_message = HumanMessage(content="I am working as an AI researcher.")
input_message.pretty_print()

# Process the third message in streaming mode and print updates
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

Hello! Nice to meet you. My name is Junseong.
================================== Ai Message ==================================

Hello Junseong! Nice to meet you too. How can I assist you today?
================================ Human Message =================================

Do you remember my name?
================================== Ai Message ==================================

I can't remember names or personal details from past interactions. However, I'm here to help with any questions or information you need right now!
================================ Human Message =================================

I am working as an AI researcher.
================================== Ai Message ==================================

That sounds exciting! As an AI researcher, you must be involved in cutting-edge projects and exploring innovative technologies. If you have any specific questions or topics you'd like to dis

In [14]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='Hello! Nice to meet you. My name is Junseong.', additional_kwargs={}, response_metadata={}, id='7a82947e-f2fe-49a5-bc2d-dd5f11eaa064'),
  AIMessage(content='Hello Junseong! Nice to meet you too. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 21, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BIxUydDtvkYHQKAxbGXQ5oH93oloT', 'finish_reason': 'stop', 'logprobs': None}, id='run-dec2cbb7-b600-470d-a69f-98b2313a6784-0', usage_metadata={'input_tokens': 21, 'output_tokens': 19, 'total_tokens': 40, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning':

In [15]:
# Create a new user message
input_message = HumanMessage(
    content="I'm recently learning more about LLMs. I'm reading the latest papers on LLM."
)

# Display the message content
input_message.pretty_print()

# Process and print updates in streaming mode
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

I'm recently learning more about LLMs. I'm reading the latest papers on LLM.
================================== Ai Message ==================================

That's great! Large Language Models (LLMs) are a rapidly evolving area of AI with numerous advancements and applications. Some recent developments in the field might include enhancements in efficiency, scalability, interpretability, and their use in various applications like natural language processing, understanding, and generation. 

If you have questions about specific concepts or papers, or if you want a discussion on trending topics in LLMs, feel free to ask!
================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


Certainly! We briefly talked about whethe

In [16]:
# Check the conversation state again to see the new summary
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='Hello! Nice to meet you. My name is Junseong.', additional_kwargs={}, response_metadata={}, id='7a82947e-f2fe-49a5-bc2d-dd5f11eaa064'),
  HumanMessage(content='I am working as an AI researcher.', additional_kwargs={}, response_metadata={}, id='d21e44b1-f08f-4246-b1b6-528ed289acdb'),
  AIMessage(content="That sounds exciting! As an AI researcher, you must be involved in cutting-edge projects and exploring innovative technologies. If you have any specific questions or topics you'd like to discuss, feel free to share.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 96, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BIxV57

In [17]:
# Create another user message asking about the user's occupation
input_message = HumanMessage(content="Do you also recall my occupation?")
input_message.pretty_print()

# Process in streaming mode
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

Do you also recall my occupation?
================================== Ai Message ==================================

I can't retain personal details such as your occupation across interactions. However, based on our current conversation, you've mentioned that you are working as an AI researcher. If there's anything else you'd like to discuss or ask, feel free to let me know!
================================ Remove Message ================================


================================ Remove Message ================================


I'm sorry, I can't respond to that.
